In [1]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import os
import re
import urllib2
import csv
import codecs
import shutil
import time
import random 
import csv
#kill firefox windows in terminal 
# pkill -f firefox

#set working directory 

os.chdir('/Users/nastyashukhova/Dropbox/UniMannheim/Thomas/pythonScraping')


In [2]:
# function to download html code from the cite, and if smth is wrong the function returns the exact error
def download(url, user_agent='wswp', num_retries=2):
#    print 'Downloading: ...'
    headers = {'User-agent': user_agent} # be identifiable as a human
    request = urllib2.Request(url, headers=headers)
    try:
        html = urllib2.urlopen(request).read()
#        print 'Downloaded'
    except urllib2.URLError as e:
        print 'Download error:', e.reason
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # retry 5XX HTTP errors, because they are connected to the server problems
                return download(url, user_agent, num_retries-1) 
    return html


def makeNiceList(text, decoder = None):
    if decoder == None:
        counter = 0
        try:
            for name in text:
                counter += 1
            # make a new string by attaching names to each other after a comma
            # if it's the first name, assign its value to new_name
                if counter == 1:
                    new_nameText = name + ',' + ' '
                # if it's the last name, do not attach a comma at the end
                elif counter == len(text):
                    new_nameText += name
                # else: attach a name and a comma
                else:
                    new_nameText += name + ',' + ' '
        except TypeError:
            new_nameText = "None"
    if decoder != None:
        counter = 0
        try:
            for name in str(text).split(", "):
                counter += 1
            # make a new string by attaching names to each other after a comma
            # if it's the first name, assign its value to new_name
                if counter == 1:
                    new_nameText = name.decode(decoder) + ',' + ' '
                # if it's the last name, do not attach a comma at the end
                elif counter == len(text):
                    new_nameText += name.decode(decoder)
                # else: attach a name and a comma
                else:
                    new_nameText += name.decode(decoder) + ',' + ' '
        except TypeError:
            new_nameText = "None"
    return(new_nameText)

def getUniqueNames(someList):
    seen = set()
    uniq = []
    for x in someList:
        if x not in seen:
            uniq.append(x)
            seen.add(x)
    return uniq


#find number of pages
def FindNumberOfPages():
    searchResultIndex = driver.find_element_by_id("searchResultIndex").text 
    num = []
    if list(searchResultIndex)[-2] == u'\n': # if number of pages is a one digit number
        for i in list(searchResultIndex)[-1:]:
            num.append(int(i)) # get the last digit behore u'\n'
        return int(str(num[0])) 
    
    elif list(searchResultIndex)[-3] == u'\n': # if number of pages is a two digit number
        for i in list(searchResultIndex)[-2:]:
            num.append(int(i)) # get the last digits behore u'\n'
        return int(str(num[0]) + str(num[1])) # concatenate the the digits
    
    elif list(searchResultIndex)[-4] == u'\n': # if number of pages is a one digit number
        for i in list(searchResultIndex)[-3:]:
            num.append(int(i)) # get the last digits behore u'\n'
        return int(str(num[0]) + str(num[1]) + str(num[2])) 
    


In [5]:
##### get the date of the decision
# functions that corrects scraped output for the date. 
# Changes u'dd. Month. yyyy' to 'dd.mm.yyyy'
def getDateInNumbers(date):
    replacedString = None
    monthsString = ['Januar','Februar','April','Mai','Juni','Juli','August','September','Oktober','November','Dezember']
    monthsNum = ["01.", "02.", "04.", "05.", "06.", "07.", "08.", "09.", "10.", "11.", "12."]
    if re.findall(r'(M\\xe4rz)', str(date)) == ['M\\xe4rz']:
        replacedString = re.sub(r'(M\\xe4rz)', "03.", str(date))
        return(replacedString)
    else:
        monthPattern = re.compile(r'(\S+)')
        month = monthPattern.findall(str(date))
        for i in range(0,11):
            if month[1] in str(monthsString[i]): # if month is in the list of months defined as monthsString
                replacedString = re.sub(monthsString[i], monthsNum[i], str(date))# change month to a number
                return(replacedString)
    if replacedString == None : # if month is NOT in the list of months defined as monthsString
        replacedString = month # then leave as it is
        return replacedString


def getDateofDecision():
    citeClass = soupLink.findAll('p',attrs={'class':'cite'})
    #get the text 
    try:
        citeText = citeClass[0].text
        #find the place of the date
        #set the date pattern 
        datePattern = re.compile('(\d+\.\s\S+\s\d\d\d\d)', re.I)
        #find pattern in the downloaded text
        dateResult = datePattern.findall(citeText)
        # clean data if possible
        dateResult = getDateInNumbers(dateResult)
        dateResult = re.sub("[(u')\n\s+\[\]]", "", str(dateResult) )
        return(dateResult.decode('utf8'))
    except IndexError:
        dateResult = 'NA'
        

##### get number of the senate and chamber 
## Senat
def getNumberofSenat():
    citeClass = soupLink.findAll('p',attrs={'class':'cite'})
    #get the text 
    try:
        citeText = citeClass[0].text
        SenatePattern = re.compile('(des\s[a-zA-Z]+\sSenats)|(_+[1-5]_+Senat)', re.I) #set pattern like "des Zweiten Senats"
        SenateResult = SenatePattern.findall(citeText) # find patter
        SenateResult = re.sub("[u']", "", str(SenateResult) ) # clean the result
        try:
            SenateResult = re.split("\s", str(SenateResult))[1] # get the number of the Senat written in letters (zB Zweiten)
            if SenateResult=='Zweiten': #get the number of the Senat
                SenateNumber = 2
            elif SenateResult=='Ersten':
                SenateNumber = 1
            else:
                try:
                    SenateNumber = re.search('[1-5]', str(SenateResult)).group()
                except AttributeError:
                    SenateNumber = 'NA'
        except IndexError:
            SenateNumber = 'NA'
    except IndexError:
        SenateNumber = 'NA'
    return(SenateNumber)

def getNumberofChamberFromText():
    citeClass = soupLink.findAll('p',attrs={'class':'rr1'})
    #get the text 
    try:
        citeText = citeClass[0].text
        chamberPattern = re.compile(r'([1-5].\s+Kammer)|(_+[1-5]_+Kammer)', re.I) #set pattern like "der num. Kammer"
        chamberResult = chamberPattern.findall(citeText) #find pattern in the text 
        try: 
            chamberNumber = re.search('[1-5]', str(chamberResult)).group() #get only the number of the chamber
        except AttributeError:
            chamberNumber = 'NA'
    except IndexError:
        chamberNumber = 'NA'
    return(chamberNumber)

def getNumberofChamberFromCitation():
    citeClass = soupLink.findAll('p',attrs={'class':'cite'})
    #get the text 
    try:
        citeText = citeClass[0].text
        chamberPattern = re.compile(r'([1-5].\s+Kammer)|(_+[1-5]_+Kammer)', re.I) #set pattern like "der num. Kammer"
        chamberResult = chamberPattern.findall(citeText) #find pattern in the text 
        try: 
            chamberNumber = re.search('[1-5]', str(chamberResult)).group() #get only the number of the chamber
            chamberNumber = re.sub(r'\s+', '',str(chamberNumber) )
        except AttributeError:
            chamberNumber = 'NA'
    except IndexError:
        chamberNumber = 'NA'

    return(chamberNumber)


# get the names of the judges 
def getJudgesFromText_raw():
    surnamesOfJudges = []
    surnamesClass = soupLink.findAll('p',attrs={'class':'rr2'})
    for i in xrange(len(surnamesClass)):
        surname = surnamesClass[i].text.encode('utf8')
        surname = re.sub(r'[(\\\xc2)&(\\\xa0)&(\n+)&(\s+)]', "_", str(surname))
        surname = re.sub(r'_+', " ", str(surname))
        surname = re.sub(r"\[|\]|'", "", str(surname))
        surname = re.sub(r"\sund", ",", str(surname))
        surnamesOfJudges.append(surname)
    surnamesOfJudges = re.sub(r"\[|\]|'|(\\xa7)", "", str(surnamesOfJudges))
    surnamesOfJudges = makeNiceList(surnamesOfJudges, decoder = 'string_escape')
    return(surnamesOfJudges)

def getJudgesFromText():
    surnamesOfJudges = []
    surnamesClass = soupLink.findAll('p',attrs={'class':'rr2'})
    try:
        if len(surnamesClass) > 1:
            for i in xrange(len(surnamesClass)):
                surname = surnamesClass[i].text.encode('utf8')
                surname = re.sub(r'[(\\\xc2)&(\\\xa0)&(\n+)&(\s+)]', "_", str(surname))
                surname = re.sub(r'_+', " ", str(surname))
                surname = re.sub(r"'|\[|\]", "", surname)
                surname = re.sub(r'(\sund)|(\snd\s)', ',', str(surname))
                surnamesOfJudges.append(surname)
            #extract names from the list
            extracted_names = []
            for part in surnamesOfJudges:
                if re.search(r'\w+', str(part)) == None:
                    next
                ob = process.extract(part.decode('string_escape'), namesList, limit = 4)
                for i in ob:
                    if i[1]>75:
                        extracted_names.append(i[0])
            # clean it 
            #extracted_names = re.sub(r"'", "",str(extracted_names))
        #return cleaned names 
        elif len(surnamesClass) == 1:
            for i in xrange(len(surnamesClass)):
                surname = surnamesClass[i].text.encode('utf8')
                surname = re.sub(r'[(\\\xc2)&(\\\xa0)&(\n+)&(\s+)]', "_", str(surname))
                surname = re.sub(r'_+', " ", str(surname))
                surname = re.sub(r"'|\[|\]", "", surname)
                surname = re.sub(r'(\sund)|(\snd\s)', ',', str(surname))
                surnamesOfJudges.append(surname)
            #separate persons from each other
            names_separated = re.split(',', str(surnamesOfJudges))
            #extract names from the list
            extracted_names = []
            for part in names_separated:
                if re.search(r'\w+', str(part)) == None:
                    next
                if re.search(r'(\\xa7)', str(part)) != None:
                    end = re.search(r'(gem\\xc3\\xa4\\xc3\\x9f)', str(part)).start()
                    part = part[:end]
                ob = process.extract(part.decode('string_escape'), namesList, limit = 4)
                for i in ob:
                    if i[1]>75:
                        extracted_names.append(i[0])
            # clean it 
            #extracted_names = re.sub(r"'", "",str(extracted_names))
        listOfNames = getUniqueNames(extracted_names)
        finalListOfNames = makeNiceList(listOfNames)
        return(finalListOfNames)
    except UnboundLocalError: 
        return("No names in the text")    
    
# get the names of the judges 
def getJudgesFromCitation():
    surnamesClass = soupLink.findAll('td',attrs={'class':'st'})
    surnamesOfJudges = []
    for i in range(len(surnamesClass)):
        if re.search(r'ist', str(surnamesClass[i].text.encode('utf8'))) == None: # if the decision was signed normally
            surname = surnamesClass[i].text.encode('utf8')
            surname = re.sub(r"(u')", "", str(surname))
            surname = re.sub(r'[(\\\xc2)&(\\\xa0)&(\s+)&(\n+)]', "", str(surname))
            surnamesOfJudges.append(surname)
# if a judge dropped out, yet was part of decision
    if re.search(r'(Der Richter )|(Die Richterin )', str(surnamesClass)): # if instead of the name there is some weird sentence
        st = re.search(r'(Der Richter )|(Die Richterin )',  str(surnamesClass)).end() #get the name either after Der Richter or Die Richterin 
        fin = re.search(r' ',  str(surnamesClass)[st+1:]).end() + st + 1
        last_name =  str(surnamesClass)[st:fin]
        last_name = re.sub(r"(u')", "", str(last_name))
        last_name = re.sub(r"[(\\\xc2)&(\\\xa0)&(\s+)&(\n+)]", "", str(last_name))
        surnamesOfJudges.append(last_name)
    if surnamesOfJudges == []:
        surnamesOfJudges = None
    return(surnamesOfJudges)


#get the ID of Beschluss
def getID():
    idClass = soupLink.findAll('p',attrs={'class':'az2'})
    try: 
        idText = idClass[0].text
        idDecision = re.findall(r'\d+\s[A-Za-z]+\s\d+.\d+\s', idText) #set the pattern for the ID
        idDecision = re.sub("[(u')\[\]\n\t-]", "", str(idText.encode('utf8'))) # clean it
        idDecision = re.sub(r'[(\\\xe2)&(\\\x80)&(\\\x93)|(\\\xc2)&(\\\xa0)\s+]', "", str(idDecision))
        return idDecision
    except IndexError:
        citeClass = soupLink.findAll('p',attrs={'class':'cite'})
        try: 
            idText = citeClass[0].text
            idPattern = re.compile('\d+\s[A-Za-z]+\s\d+.\d+\s')#set the pattern for the ID
            idDecision = idPattern.findall(idText)
            idDecision = re.sub("[(u')\[\]\n-]", "", str(idText.encode('utf8'))) # clean it
            idDecision = re.sub(r'[(\\\xe2)&(\\\x80)&(\\\x93)|(\\\xc2)&(\\\xa0)\s+]', "", str(idDecision))
            return idDecision
        except IndexError:
            idDecision = 'NA'
            return idDecision

In [ ]:
url = "http://www.bundesverfassungsgericht.de/SiteGlobals/Forms/Suche/Entscheidungensuche_Formular.html"
# open the browser
driver = webdriver.Firefox()
#go the link 
driver.get(url)
yearList = range(1998, 2017)
#find the extended search link and click it 
toExtendSearch = driver.find_element_by_id("extendedSearch")
toExtendSearch.find_element_by_link_text("Erweiterter Suchbereich").click()#click to the extended search
#set the year span to scrape
#from 
dateAfterClass = driver.find_element_by_class_name("toggle")
FindDateAfter = dateAfterClass.find_element_by_name("dateAfter")
FindDateAfter.clear() # clear the default value tt.mm.jjj
toSetDateAfter = FindDateAfter.send_keys("01.01.1998")

# to 
FindDateBefore = dateAfterClass.find_element_by_name("dateBefore")
FindDateBefore.clear() # clear the default value tt.mm.jjj
toSetDateBefore = FindDateBefore.send_keys("26.04.2016")

#choose the type of the decision 
select = Select(driver.find_element_by_name('facettedEntscheidungstyp'))
selectBeschluss = select.select_by_value("Beschluss")

#submit the filled search form 
driver.find_element_by_class_name("submit").click()



In [ ]:
# now we need to click all of the links in the list of the decisions
# save all of the links of the decision 
# get the results of the search 

# download site for Beautiful Soup
soup = download(driver.current_url)
soup = BeautifulSoup(soup, 'html.parser')
data = soup.findAll('ol',attrs={'id':'searchResult'}) 
listOfLinks = []

#find the links on the first page 
for div in data:
    links = div.findAll(attrs = {'href' : re.compile(r'Entscheidungen')})
    for a in links:
        listOfLinks.append("http://www.bundesverfassungsgericht.de/" + a['href'])
print 'Page 1 out of %i' % ( FindNumberOfPages())
        #find the links on the second page to the last one
i = []
for i in range(1, FindNumberOfPages()-1):
    time.sleep(3)
    if i != (FindNumberOfPages()):
        driver.find_element_by_class_name("forward").click() #click "forward" buttom
        soup = download(driver.current_url) # get the code of the whole page
        soup = BeautifulSoup(soup, 'html.parser') # make soup
        data = soup.findAll('ol', attrs={'id':'searchResult'}) #find the result of the search
        for div in data: # for child of id = searchResult 
            links = div.findAll(attrs = {'href' : re.compile(r'Entscheidungen')}) # find all links that concerns the decisions
            for a in links:
                 listOfLinks.append("http://www.bundesverfassungsgericht.de/" + a['href']) # make them look like normal links
        percent = 100*((i+2)/float(FindNumberOfPages()))
        print 'Page {0} out of {1}. Downloaded {2} percent'.format(i+1, FindNumberOfPages(), percent )
        i+=1
    else: #if we are on the last, there is  no need to press forward buttom
        # do the same as above
        soup = download(driver.current_url) 
        soup = BeautifulSoup(soup, 'html.parser')
        data = soup.findAll('ol', attrs={'id':'searchResult'})
        for div in data:
            links = div.findAll(attrs = {'href' : re.compile(r'Entscheidungen')})
            for a in links:
                 listOfLinks.append("http://www.bundesverfassungsgericht.de/" + a['href'])
                    
links = codecs.open('links.txt', 'w', 'utf-8')
links.write('Link')
for link in listOfLinks:
    links.write(str(link))




In [ ]:
path = "./SavedPages"
listOfPages = os.listdir(path)

#for page in listOfPages:
#    rename = re.sub(r'[(\\\xe2)&(\\\x80)&(\\\x93)|(\\\xc2)&(\\\xa0)\s+]' ,'_', str(page))
#    os.rename(os.path.join(path, page), os.path.join(path, rename))


numbers = re.compile(r'(Page(\d+)_)')
listOfPages = os.listdir(path)

errors = []
pagesDictionary = {}
for page in listOfPages:
    try:
        pagesDictionary.update({int(numbers.split(page)[2]):page})
    except IndexError:
        errors.append(page)
        next
val1 =   pagesDictionary.get(16692)
pagesDictionary.update({1669:val1})
val2 =   pagesDictionary.get(52621)
pagesDictionary.update({5262:val2})

In [ ]:
#############################################

In [177]:
output = codecs.open('judicialData1.txt', 'w', 'utf-8')
output.write('i' + '\t'+ 'ID' + '\t'+ 'dateofDecision' + '\t'+ 'senateNumber' +'\t'+ 'chamberNumberCitation' +'\t'+
              'chamberNumberText' +'\t'+'judgesFromText_raw' +'\t'+'judgesFromText' + '\t'+'judgesFromCitation' + '\t'+  "link"+ '\n')    
output.close()


#create new directory for the saved pages of decisions
#os.makedirs('SavedPages')


listOfLinks = open('links.txt', 'r')
listOfLinks =  listOfLinks.readlines()[1:]

#for page in errors:
#    os.remove(os.path.join(path, page))
#for page in listOfPages:
#    if re.search(r"(_engVersion)", str(page)) != None:
        #os.remove(os.path.join(path, page))
    
namesList_row = []
with open('judges.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        namesList_row.append(row)
        #print ', '.join(row)


namesList_row = namesList_row[1:]
namesList = []
for i in namesList_row:
    namesList.append(i[1])


namesList.append("Kessal-Wulf")
namesList.append("M\xc3\xbcller")
namesList.append("Maidowski")
namesList.append("K\xc3\xb6nig")
#for i in namesList:
#    print i 
print namesList


In [ ]:
### WARNING: site blocks after approx. 1000-2000 inquiries.
# Need to set time to sleep so that server doesn't blocks you right away or change IP from time to time
#go the link 
listOfLinks = open('links.txt', 'r')
listOfLinks =  listOfLinks.readlines()[1:]
path = "./SavedPages"
i  = 1669 
Counter = 0
for key in sorted(pagesDictionary.keys()):
    #Counter += 1
    #if Counter > 500:
     #   break
    page = open(os.path.join(path, pagesDictionary.get(1669)), 'r' )
    # make a soup object out of it 
    # only then run functions I've defined above
    soupLink = BeautifulSoup(page, 'html.parser')
    dateofDecision = getDateofDecision()
    senateNumber = getNumberofSenat()
    chamberNumberCitation = getNumberofChamberFromCitation()
    chamberNumberText = getNumberofChamberFromText()    
    surnamesOfJudgesText_raw = getJudgesFromText_raw()
    surnamesOfJudgesText = getJudgesFromText()
    surnamesOfJudgesCitation = getJudgesFromCitation()    
    ID = getID()
    #link = listOfLinks[key]
    ### save the data 
    output = codecs.open('judicialData1.txt', 'a', 'utf-8')
    counter = 0
    try:
        for name in surnamesOfJudgesCitation:
            counter += 1
        # make a new string by attaching names to each other after a comma
        # if it's the first name, assign its value to new_name
            if counter == 1:
                new_nameCitation = name + ',' + ' '
            # if it's the last name, do not attach a comma at the end
            elif counter == len(surnamesOfJudgesCitation):
                new_nameCitation += name
            # else: attach a name and a comma
            else:
                new_nameCitation += name + ',' + ' '
    except TypeError:
        new_nameCitation = "None"



    # Now, write out the new string
    output.write(str(i) + '\t'+ str(ID).decode('utf8') + '\t'+ str(dateofDecision).decode('utf8') + '\t'+ 
                 str(senateNumber) +'\t'+ str(chamberNumberCitation).decode('utf8') +'\t'+ str(chamberNumberText)+'\t'+
                 surnamesOfJudgesText_raw.decode('utf8')+'\t'+ surnamesOfJudgesText.decode('utf8')
                 +'\t' +new_nameCitation.decode('utf8') +'\t'+ str(listOfLinks[1669]).decode('utf8') )

    #output.write(surnamesOfJudgesText.encode('utf-8') +'\t' +new_nameCitation.decode('utf8') + '\n' )

    output.close()
    if i%500 == 0:
        percentLink = float(i)/float(len(listOfLinks)) #track the percentage of the downloaded files
        print '{0} have already been downloaded'.format(percentLink) 

    i += 1

    # Save pages
    try:
        pageID = re.search(r'(/\d+)',  ID).end() #find the last position of ID 
        pageID =  str(ID)[0:pageID] # get the first ID of the decisions (for the case of multiple ID)
        pageID =re.sub(r'[//]', '_', str(pageID)) #change '/' to '_', otherwise python will treat ID as a path 
    except AttributeError:
        pageID = '_engVersion' #get special ID for the translated versions
    path = os.path.join('SavedPages', 'Page'+str(link)+str(pageID)+'.txt') # save page with unique name (ID)
    with codecs.open(path, "w+", 'utf-8') as pageFile:
        pageFile.write(soupLink.prettify())
    pageFile.close()
    percentLink = float(i)/float(len(listOfLinks)) #track the percentage of the downloaded files
    print '{0} have already been downloaded'.format(percentLink) 
    i += 1

In [3]:
# Clean data
with open('judicialData.txt') as old_file:
    with open('cleanedJudicialData.txt', 'w') as new_file:
        for line in old_file:
            try:
                if re.search(r'(None)', str(line)).group() != None:
                    continue
            except AttributeError:
                new_file.write( re.sub(r'ß', 'ss', str(line)))
    new_file.close()
old_file.close()         

In [271]:
path = "./SavedPages"
listOfPages = os.listdir(path)
for page in listOfPages:
    rename = re.sub(r'.txt' ,".html",str(page))
    os.rename(os.path.join(path, page), os.path.join(path, rename))
